## Evolution Strategy (ES)

In [19]:
import numpy as np
import random
import glob
import os
import time
import csv
import copy

### Problem and Fitness Functions

In [20]:
def load_problem(file_path):
    """Loads a distance matrix from a .npy file."""
    return np.load(file_path)

def calculate_fitness(individual, dist_matrix):
    """Calculates the total distance of a tour (individual)."""
    fitness = 0
    num_cities = len(individual)
    for i in range(num_cities):
        city1 = individual[i]
        city2 = individual[(i + 1) % num_cities]
        fitness += dist_matrix[city1, city2]
    return fitness

### Initialization (Greedy and Random)

In [21]:
def create_greedy_individual(num_cities, dist_matrix):
    """Creates one individual using the "Nearest Neighbor" heuristic."""
    current_city = random.randrange(num_cities)
    unvisited = set(range(num_cities))
    unvisited.remove(current_city)
    tour = [current_city]
    
    while unvisited:
        nearest_city = min(unvisited, key=lambda city: dist_matrix[current_city, city])
        unvisited.remove(nearest_city)
        tour.append(nearest_city)
        current_city = nearest_city
        
    return tour

def create_random_individual(num_cities):
    """Creates one individual by randomly shuffling the city indices."""
    individual = list(range(num_cities))
    random.shuffle(individual)
    return individual

def initialize_population(pop_size, num_cities, dist_matrix, greedy_ratio=0.8):
    """Initializes the population with 80% greedy seeds and 20% random seeds."""
    population = []
    
    num_greedy = int(pop_size * greedy_ratio)
    for _ in range(num_greedy):
        population.append(create_greedy_individual(num_cities, dist_matrix))
        
    num_random = pop_size - num_greedy
    for _ in range(num_random):
        population.append(create_random_individual(num_cities))
        
    return population

### Genetic Operators (For TSP)

In [22]:
# Crossover
def order_crossover_ox(parent1, parent2):
    """Performs Order Crossover (OX)."""
    size = len(parent1)
    child = [None] * size
    start, end = sorted(random.sample(range(size), 2))
    child[start:end+1] = parent1[start:end+1]
    segment_genes = set(parent1[start:end+1])
    
    p2_genes_to_add = [gene for gene in parent2 if gene not in segment_genes]
            
    p2_idx = 0
    for i in range(size):
        child_idx = (end + 1 + i) % size 
        if child[child_idx] is None:
            child[child_idx] = p2_genes_to_add[p2_idx]
            p2_idx += 1
            
    return child

# Mutation
def swap_mutation(individual):
    """Performs swap mutation by swapping two random positions."""
    ind_copy = individual[:]
    idx1, idx2 = random.sample(range(len(ind_copy)), 2)
    ind_copy[idx1], ind_copy[idx2] = ind_copy[idx2], ind_copy[idx1]
    return ind_copy

### Evolution Strategy (ES) Solver

In [23]:
def solve_es(problem_path, mu, lambda_, generations, mutation_rate, es_mode):
    """
    Main function to run the Evolution Strategy.
    
    Args:
        problem_path (str): Path to the .npy problem file.
        mu (int): Population size (number of parents).
        lambda_ (int): Number of offspring to generate.
        generations (int): Number of generations to run.
        mutation_rate (float): Probability of mutation.
        es_mode (str): "Steady-State" for (μ+λ) or "Generational" for (μ,λ).
    """
    
    # Load problem
    dist_matrix = load_problem(problem_path)
    num_cities = dist_matrix.shape[0]
    
    # Initialization
    population = initialize_population(mu, num_cities, dist_matrix)
    fitnesses = [calculate_fitness(ind, dist_matrix) for ind in population]
    
    best_overall_fitness = min(fitnesses)
    
    # Generation Loop
    for gen in range(generations):
        
        # Generate Offspring
        offspring = []
        offspring_fitnesses = []
        for _ in range(lambda_):
            
            # Parent Selection (ES-style)
            # We select parents UNIFORMLY AT RANDOM from the μ population.
            # The selection pressure is applied later, at the survivor stage.
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            
            # Crossover
            child = order_crossover_ox(parent1, parent2)
            
            # Mutation
            if random.random() < mutation_rate:
                child = swap_mutation(child)
                
            child_fitness = calculate_fitness(child, dist_matrix)
            offspring.append(child)
            offspring_fitnesses.append(child_fitness)

        # Survivor Selection (This is the core of ES)
        
        if es_mode == "Steady-State":
            # ES(μ+λ) (elitist): Combine parents and offspring, select best μ
            combined_pop = population + offspring
            combined_fits = fitnesses + offspring_fitnesses
            
            sorted_combined = sorted(zip(combined_pop, combined_fits), key=lambda x: x[1])
            
            population = [ind for ind, fit in sorted_combined[:mu]]
            fitnesses = [fit for ind, fit in sorted_combined[:mu]]
            
        elif es_mode == "Generational":
            # ES(μ,λ) (non-elitist): Select best μ from offspring
            sorted_offspring = sorted(zip(offspring, offspring_fitnesses), key=lambda x: x[1])
            
            population = [ind for ind, fit in sorted_offspring[:mu]]
            fitnesses = [fit for ind, fit in sorted_offspring[:mu]]

        # Update the best-ever fitness found
        current_best_fitness = fitnesses[0] # Population is sorted
        if current_best_fitness < best_overall_fitness:
            best_overall_fitness = current_best_fitness
            
    return best_overall_fitness

### Main Execution Block

In [24]:
# PARAMETERS
ES_MODE = "Steady-State"
# ES_MODE = "Generational" 

MU = 30               # Population size (μ)
LAMBDA = 60             # Offspring size (λ)
GENERATIONS = 500
MUTATION_RATE = 0.3   

# File Paths
PROBLEM_FOLDER = "lab02"
RESULTS_FOLDER = "CSV Results"
# End of Parameters

os.makedirs(RESULTS_FOLDER, exist_ok=True)

if ES_MODE == "Generational" and LAMBDA < MU:
    raise ValueError(f"For 'Generational' mode (μ,λ), LAMBDA ({LAMBDA}) must be >= MU ({MU}).")
elif ES_MODE not in ["Steady-State", "Generational"]:
    raise ValueError(f"Unknown ES_MODE: '{ES_MODE}'. Must be 'Steady-State' or 'Generational'.")

if ES_MODE == "Steady-State":
    output_file = os.path.join(RESULTS_FOLDER, "es_steady_state.csv")
    print(f"Running ES(μ+λ) 'Steady-State' Mode...")
else:
    output_file = os.path.join(RESULTS_FOLDER, "es_generational.csv")
    print(f"Running ES(μ,λ) 'Generational' Mode...")

print(f"Params: μ={MU}, λ={LAMBDA}, Gens={GENERATIONS}, Mut_Rate={MUTATION_RATE}\n")

problem_files = glob.glob(os.path.join(PROBLEM_FOLDER, "*.npy"))
problem_files.sort() 

results = []

print(f"{'File Name':<20} | {'Best Fitness':<15} | {'Time (s)':<10}")
print("-" * 49)

for file_path in problem_files:
    file_name = os.path.basename(file_path)
    
    start_time = time.time()
    
    best_fitness = solve_es(
        problem_path=file_path,
        mu=MU,
        lambda_=LAMBDA,
        generations=GENERATIONS,
        mutation_rate=MUTATION_RATE,
        es_mode=ES_MODE
    )
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    result_row = {
        "file_name": file_name,
        "best_fitness": best_fitness,
        "time": elapsed_time
    }
    results.append(result_row)
    
    print(f"{file_name:<20} | {best_fitness:<15.2f} | {elapsed_time:<10.4f}")

# Save results to CSV
try:
    with open(output_file, 'w', newline='') as f:
        fieldnames = ["file_name", "best_fitness", "time"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        
        writer.writeheader()
        writer.writerows(results)
        
    print(f"\n Success! Results saved to '{output_file}'")

except PermissionError:
    print(f"\n ERROR: Permission denied. Could not write to '{output_file}'.")
except Exception as e:
    print(f"\n ERROR: An unexpected error occurred while writing CSV: {e}")

Running ES(μ+λ) 'Steady-State' Mode...
Params: μ=30, λ=60, Gens=500, Mut_Rate=0.3

File Name            | Best Fitness    | Time (s)  
-------------------------------------------------
problem_g_10.npy     | 1497.66         | 0.1552    
problem_g_100.npy    | 4324.67         | 0.7088    
problem_g_1000.npy   | 14146.10        | 7.8079    
problem_g_20.npy     | 1755.51         | 0.2173    
problem_g_200.npy    | 6261.33         | 1.3327    
problem_g_50.npy     | 2879.78         | 0.4005    
problem_g_500.npy    | 9653.96         | 3.5275    
problem_r1_10.npy    | 184.27          | 0.1595    
problem_r1_100.npy   | 765.23          | 0.7047    
problem_r1_1000.npy  | 2588.51         | 7.7830    
problem_r1_20.npy    | 350.38          | 0.2263    
problem_r1_200.npy   | 1128.15         | 1.3656    
problem_r1_50.npy    | 572.34          | 0.4081    
problem_r1_500.npy   | 1765.16         | 3.5604    
problem_r2_10.npy    | -411.70         | 0.1587    
problem_r2_100.npy   | -4671.90    